<a href="https://colab.research.google.com/github/rebeccaxchu/Real-Property-Assessments/blob/main/scrips/extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# import necessary libraries
!pip install azure-storage-blob
!pip install sodapy
import pandas as pd
import numpy as np
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from io import StringIO
from sodapy import Socrata

In [13]:
#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
# Read Original Data
client = Socrata("opendata.maryland.gov", None)
results = client.get("kb22-is2w", limit=350000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [15]:
df_raw=results_df

In [16]:
df_raw.dropna()

,land_use_code_mdp_field_lu_desclu_sdat_field_50,new_construction_data_county_assessment_credit_sdat_field_192,parent_account_number_account_number_sdat_field_388,additional_c_a_m_a_data_dwelling_construction_code_mdp_field_strucnst_sdat_field_263,last_activity_date_yyyy_mm_dd_sdat_field_392,base_cycle_data_improvements_value_sdat_field_155,base_cycle_data_preferential_land_value_sdat_field_156,parent_account_number_district_ward_sdat_field_387,special_dates_and_data_inchoate_lien_date_yyyy_mm_dd_sdat_field_209,record_key_owner_occupancy_code_mdp_field_ooi_sdat_field_6,...,section_mdp_field_section_sdat_field_39,premise_address_direction_mdp_field_premsdir_sdat_field_22,special_dates_and_data_non_perc_area_unit_mdp_field_nprcluom_sdat_field_220,base_cycle_data_assessor_code_sdat_field_160,premise_address_number_suffix_sdat_field_21,record_key_subdistrict_sdat_field_4,municipal_tax_class_sdat_field_59,sales_segment_1_grantor_deed_reference_1_clerk_initials_mdp_field_gr1clrk1_sdat_field_81,special_dates_and_data_critical_area_code_mdp_field_crtarcod_sdat_field_223,sales_segment_2_grantor_deed_reference_1_clerk_initials_sdat_field_101


In [17]:
df_raw.shape

(352228, 209)

In [18]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352228 entries, 0 to 35227
Columns: 209 entries, land_use_code_mdp_field_lu_desclu_sdat_field_50 to sales_segment_2_grantor_deed_reference_1_clerk_initials_sdat_field_101
dtypes: object(209)
memory usage: 56.2+ MB


In [19]:
df_raw.columns

Index(['land_use_code_mdp_field_lu_desclu_sdat_field_50',
       'new_construction_data_county_assessment_credit_sdat_field_192',
       'parent_account_number_account_number_sdat_field_388',
       'additional_c_a_m_a_data_dwelling_construction_code_mdp_field_strucnst_sdat_field_263',
       'last_activity_date_yyyy_mm_dd_sdat_field_392',
       'base_cycle_data_improvements_value_sdat_field_155',
       'base_cycle_data_preferential_land_value_sdat_field_156',
       'parent_account_number_district_ward_sdat_field_387',
       'special_dates_and_data_inchoate_lien_date_yyyy_mm_dd_sdat_field_209',
       'record_key_owner_occupancy_code_mdp_field_ooi_sdat_field_6',
       ...
       'section_mdp_field_section_sdat_field_39',
       'premise_address_direction_mdp_field_premsdir_sdat_field_22',
       'special_dates_and_data_non_perc_area_unit_mdp_field_nprcluom_sdat_field_220',
       'base_cycle_data_assessor_code_sdat_field_160',
       'premise_address_number_suffix_sdat_field_21',


In [23]:
df_cleaned=df_raw.copy()

In [24]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352228 entries, 0 to 352227
Columns: 209 entries, land_use_code_mdp_field_lu_desclu_sdat_field_50 to sales_segment_2_grantor_deed_reference_1_clerk_initials_sdat_field_101
dtypes: object(209)
memory usage: 56.2+ MB


In [ ]:

# Initialize an empty list to hold the names of columns to drop
columns_to_drop = []

# Loop through each column in the DataFrame
for column in df_raw.columns:
    # Calculate the percentage of NaN values in the current column
    percent_missing = df_raw[column].isnull().mean() * 100

    # If more than 70% of the values in the column are NaN, add it to the list
    if percent_missing > 70:
        columns_to_drop.append(column)

# Create a new DataFrame with just the columns to drop
reject_table = df_raw[columns_to_drop]

# Drop these columns from the original DataFrame
df_cleaned = df_raw.drop(columns=columns_to_drop)

# 'df_cleaned' is now your DataFrame without the columns where more than 70% of the values are NaN
# 'reject_table' contains the columns that were removed.


In [25]:
df_cleaned.shape

(352228, 209)

In [27]:
# specify the path to your JSON configuration file
config_file_path= '/content/gdrive/MyDrive/Colab Notebooks/mdproperty/credentials.json'

# load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

Connection_String = config["ConnectionString"]
CONTAINER_AZURE= 'mdrealproperty'
blob_name='property.csv'

In [28]:
# Convert DataFrame to CSV
output=StringIO()
df_raw.to_csv(output,index=False)
data = output.getvalue()
output.close()

In [29]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()


In [30]:
azure_upload_blob(Connection_String, CONTAINER_AZURE, blob_name, data)

Uploaded to Azure Blob: property.csv
